[View in Colaboratory](https://colab.research.google.com/github/jcnrn/f18_ds_nlp/blob/master/conranTextSimilarityJacardianCosine.ipynb)

# Homework 2

Introduction: The purpose of this homework will be to examine the similarity of a number of articles contained in the data directory. Specifically, the intent is to implement different document similarity techniques and then see how similar these documents are using said techniques. Finally, we would like to present the similarity in a table or heatmap.

Step 1:
In order to begin this exercise, we will first need to iterate over the data directory and read the content of each file. We will store the data from each file in a hash.

In [0]:
import os
import nltk.data
import numpy
from os import listdir
from os.path import isfile, join
from nltk.tokenize import TreebankWordTokenizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
treebank_tokenizer = TreebankWordTokenizer()
import itertools
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
import re, math
from collections import Counter
import pandas as pd

from nltk.corpus import stopwords

print(os.getcwd())
os.chdir('C:/Users/JEC/Desktop/hw2/data/')
print(os.getcwd())

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JEC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\JEC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\JEC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\JEC\Desktop\hw2
C:\Users\JEC\Desktop\hw2\data


In [0]:
import os
os.getcwd()

'C:\\Users\\JEC\\Desktop\\hw2'

In [0]:
#path_to_homework_5_data_directory = "C:/Users/JEC-Work/Desktop/hw2/data"

#article_hash = {} # this hash should serve to represent the content of the files in the data directory
# use the filename as the hash key and the value will be the text of the file
# thus you would be able to retrieve an individual documents text like: article_hash["article_1"]

# here we will get a list of the filenames of things contained in the data directory
#files = [f for f in listdir(path_to_homework_5_data_directory) if isfile(join(path_to_homework_5_data_directory, f))]

# here you will iterate over all the files contained in the directory
#for f in files:
#        file_texts += read_file(join(directory, f) )

# implement the rest of this line and the content of the loop
    # you will read the content of the file and put the text into the hash
    # article_hash[filename_variable] = the content of the file



#get list of files in directory
file = []
file = os.listdir("C:/Users/JEC/Desktop/hw2/data")

#create list of text from files
text = []
for i in file:
    text.append(open(i, encoding='utf-8').read())

#create dictionary of the two
article_hash= dict(zip(file,text))
#article_hash['article_9']                        #test
article_hash.keys()

dict_keys(['article_1', 'article_10', 'article_11', 'article_12', 'article_2', 'article_3', 'article_4', 'article_5', 'article_6', 'article_7', 'article_8', 'article_9'])

Step 2:
Text processing. Now that you have the content of the files read into a hash you will be able to process them. Specifically, you should perhaps employ sentence segmentation, tokenization, and stemming to get a new representation of the document. Here you will want to build a sufficiently flexible approach so that you can try out several different pre-processing strategies to see how it affects your similarity scores.

We'll create a new hash that contains the processed text.

In [0]:
# setup a new hash to store the results in
processed_article_hash = {}

# iterate through the keys, i.e. document ids, in the hash to pull out the stored text and process
for key in article_hash.keys():
    text_of_article = article_hash[key]
    
    text_of_article = treebank_tokenizer.tokenize(text_of_article)
    text_of_article = [word for word in text_of_article if word not in stopwords.words('english')]
    text_of_article = [wordnet_lemmatizer.lemmatize(word) for word in text_of_article]
    
    # here you can apply your segmentation, tokenization, and stemming steps as you see fit
    
    processed_article_hash[key] = text_of_article
    
processed_article_hash["article_12"]    #test

['Two',
 'D.C.',
 'Metro',
 'employee',
 'charged',
 'stealing',
 'thousand',
 'fare',
 'http',
 ':',
 '//www.facebook.com/dana.hedgpeth',
 '8-10',
 'minute',
 'Metro',
 'employee',
 'Horace',
 'Dexter',
 'McDade',
 'typically',
 'spent',
 'day',
 'traveling',
 'one',
 'subway',
 'station',
 'another',
 ',',
 'collecting',
 'bill',
 'coin',
 'fare',
 'machines.',
 'Often',
 ',',
 'Metro',
 'Transit',
 'Police',
 'officer',
 'John',
 'Vincent',
 'Haile',
 'nearby',
 ',',
 'watching',
 'McDade',
 'made',
 'round',
 'trucked',
 'cash',
 'collection',
 'facility',
 'Alexandria.',
 'But',
 'authority',
 'allege',
 'two',
 'men',
 'taking',
 'unofficial',
 'detour',
 ',',
 'driving',
 'spot',
 'near',
 'Capital',
 'Beltway',
 'hundred',
 'yard',
 'collection',
 'building',
 'hiding',
 'part',
 'day',
 '’',
 'take.',
 'When',
 'shift',
 'done',
 ',',
 'would',
 'return',
 'car',
 'make',
 'money.',
 'On',
 'Wednesday',
 'night',
 ',',
 'federal',
 'authority',
 'arrested',
 'pair',
 'charged'

Step 3:
Implement two similarity techniques. 
We would like to examine the Jacard Similarity and Cosine.

Jacardian Similarity: here we want to identify the set of words in two documents that overlap and then divide that by the count of unique words across both documents.

Cosine Similarity: Here we want to create vector representations for each document. Specifically, we want to come up with a vector that is based on the list of all words that occur across both documents. Then for each document we will create a vector that includes the counts of the number of time a word occurs in the document.

So if the document 1 is: "the ship sails at midnight" and document 2 is: "the crow flies at noon." We would be creating a vector like: [the, ship, sails, at, midnight, crow, flies, noon]. Then we would calculate the values of the vector for each document. For document 1: [1,1,1,1,1,0,0,0] and for document 2: [1,0,0,1,0,1,1,1]. With these two vectors we would simply take the dot product and that would provide the cosine similarity. 


In [0]:
#just to convert to string
def stringify(whatever):
    newString = " ".join(whatever)
    return newString

#doc1 = stringify(processed_article_hash[key])
#type(doc1)
    

In [0]:
def jacardian_distance(document_1_data, document_2_data):
    doc1String = stringify(document_1_data)
    doc2String = stringify(document_2_data)
    words_in_doc_1_not_in_doc_2 = set(doc1String.split()) 
    words_in_doc_2_not_in_doc_1 = set(doc2String.split())
    words_in_both_doc_1_and_doc_2 = words_in_doc_1_not_in_doc_2.intersection(words_in_doc_2_not_in_doc_1)
    
    jacardian = float(len(words_in_both_doc_1_and_doc_2)) / (len(words_in_doc_1_not_in_doc_2) + len(words_in_doc_2_not_in_doc_1) - len(words_in_both_doc_1_and_doc_2))
    
    return jacardian

#test
doc1=processed_article_hash['article_1']
doc2=processed_article_hash['article_12']
jacardian_distance(doc1, doc2)

0.06680805938494168

In [0]:
#no clue how to vectorize documents against the combined
#def cosine_similarity(document_1_data, document_2_data):
    #document_vector_word_index = [document_1_data + document_2_data]
    #document_vector_word_index = list(set(document_vector_word_index))
   # print(document_vector_word_index)
    #document_1_vector = [] # fill in the array with the frequency of the words in the document
    #document_2_vector = [] # fill in the array with the frequency of the words in the document

    
#adapted from https://stackoverflow.com/questions/15173225/calculate-cosine-similarity-given-2-sentence-strings?rq=1

def cosine_similarity(document_1_data, document_2_data):
    WORD = re.compile(r'\w+') 
    
    def get_cosine(vec1, vec2):
        intersection = set(vec1.keys()) & set(vec2.keys())
        numerator = sum([vec1[x] * vec2[x] for x in intersection])
        sum1 = sum([vec1[x]**2 for x in vec1.keys()])
        sum2 = sum([vec2[x]**2 for x in vec2.keys()])
        denominator = math.sqrt(sum1) * math.sqrt(sum2)
        
        if not denominator:
            return 0.0
        else:
            return float(numerator) / denominator
        
    def text_to_vector(text):
        words = WORD.findall(text)
        return Counter(words)
    
    text1 = stringify(document_1_data)
    text2 = stringify(document_2_data)
    
    vector1 = text_to_vector(text1)
    vector2 = text_to_vector(text2)
    
    cosine = get_cosine(vector1, vector2)
    
    return(cosine)

    
#test   
doc1=processed_article_hash['article_6']
doc2=processed_article_hash['article_12']  
cosine_similarity(doc1, doc2)

0.16454108484552787

Step 4:
Now that we have our two similarity measures, we want to examine each document relative to each other and calculate their similarity. 

So we will want to create two tables that show the document similarities using both techniques.

In [0]:
#i have no idea how to do it this way
# create a variable to store your table data... you could use a hash or some other data structure. 
# We just want it to identify which document is being compared to which other document.

#data_structure_for_jacard_similarity = #
#data_structure_for_cosine_similarity = #

#for doc_1_key in article_hash.keys():
   # for doc_2_key in article_hash.keys():
        # we have the nested for loops as one way to compare each document to each other document
     #   data_structure_for_jacard_similarity[doc_1_key][doc_2_key] = jacardian_distance(doc_1_processed_text, doc_2_processed_text)
      #  data_structure_for_cosine_similarity[doc_1_key][doc_2_key] = jacardian_distance(doc_1_processed_text, doc_2_processed_text
                                                                                       

# finally, find some way to present this data back. Either as a straight table or a heatmap.

df = pd.DataFrame()
dataList = []
for doc_1_key in article_hash.keys():
   for doc_2_key in article_hash.keys():
    temp = (doc_1_key, doc_2_key, jacardian_distance(processed_article_hash[doc_1_key],processed_article_hash[doc_2_key]),cosine_similarity(processed_article_hash[doc_1_key],processed_article_hash[doc_2_key]))
    dataList.append(temp)


#textfile.close()
df=pd.DataFrame(dataList,columns=['First','Second','Jacard','Cosine'])
df.head()


,First,Second,Jacard,Cosine
0,article_1,article_1,1.000000,1.000000
1,article_1,article_10,0.045296,0.068519
2,article_1,article_11,0.034358,0.077635
3,article_1,article_12,0.066808,0.125983
4,article_1,article_2,0.134402,0.474544


In [0]:
df.to_csv('articleSimilarities.csv', sep=',', encoding='utf-8')

Step 5:
You should now have two different similarity mechanisms. What do your results suggest? From perusing the documents, do you think the suggested ones are similar or not? Does tokenization, stemming, stop word removal or anything else improve your results?

Write a brief description of your reactions to identifying these similar documents and what measures and pre-processing steps you think worked best.

JEC Response:

In comparing these documents, I noticed most strongly that removing stop words moved the similarity in documents for most from around a .70-.80 to about a .05-.25 result in similarity with the highest achieving .28 in Jacard and .61 in Cosine. I personally feel that lemmatization might be better at capturing similarity in feel among documents, but I'm unsure if this is the correct approach. 

As mentioned above the .28 Jacard and .61 Cosine is between Article 11 and Article 10. These two articles are about the DC Metro and do share much similarity. In second with a Jacard of .16 and a Cosine of .42 are articles 5 and 6 which discuss Supreme Court Actions. While the articles vary about on what the Supreme Court is acting, they do have commonality. In the middle, Article 7 and 2 have a .19 cosine similarity, but these two documents are about the Superbowl and Chernobyl. Articles 7 and 4 with a .13 in Jacardian and a .27 in Cosine with the two articles discussing the Superbowl and the World Series victories which is at least in the realm of sports, but still differs. At the low end the documents are indeed strongly dissimilar. Overall, it seems cosine similarity is a better indicator of similarity for this exercise

Joseph Conran



<-- put your comments here -->